# Glands Segmentator #


In this file, I use two models that I created to successfully perform gland segmentation. The process forms a pipeline where an input image is provided for segmentation, and the output is an image with the segmented glands.

# Procesing of big images #

Imports:

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import tifffile

Loading models:

In [43]:
import unet_core 
print(unet_core.__file__)  # Pokazuje lokalizację faktycznego pliku
dir(unet_core)         

# Czy zawiera 'UNET'?


C:\Users\stszy\miniconda3\Lib\site-packages\unet_core\__init__.py


['UNet',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'dataset',
 'model',
 'utils']

In [50]:
detectionModel = YOLO("../model/saved_models/Glands_Finder_Augumented_Data_best.pt")
from unet_core.unet_interface import UNET
modelUNET=UNET._model(name="GlandsFInder")

Model with name GlandsFInder doesn't exist


 Helper function:

In [ ]:
def tile_image(image, tile_size, overlap):
    tiles = []
    positions = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    for y in range(0, h, stride):
        for x in range(0, w, stride):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y))
    return tiles, positions


def detect_on_tiles(tiles, positions, tile_size,model):
    all_detections = []
    for tile, (x_offset, y_offset) in zip(tiles, positions):
        results = model(tile, conf=0.6)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1 += x_offset
            x2 += x_offset
            y1 += y_offset
            y2 += y_offset
            conf = float(box.conf)
            cls = int(box.cls)
            all_detections.append((x1, y1, x2, y2, conf, cls))
    return all_detections


def draw_detections(image, detections, class_names):
    annotated = image.copy()
    for x1, y1, x2, y2, conf, cls in detections:
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"{class_names[cls]} {conf:.2f}"
        cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return annotated


def merge_overlapping_boxes_hierarchical(
        detections,
        iou_threshold=0.5,
        containment_thresh=0.8,
        center_inclusion=False,
        linkage='complete'
):
    def iou(boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        interW = max(0, xB - xA)
        interH = max(0, yB - yA)
        interArea = interW * interH
        if interArea == 0:
            return 0.0
        areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
        return interArea / float(areaA + areaB - interArea)

    def containment_ratio(inner, outer):
        xA = max(inner[0], outer[0])
        yA = max(inner[1], outer[1])
        xB = min(inner[2], outer[2])
        yB = min(inner[3], outer[3])
        interW = max(0, xB - xA)
        interH = max(0, yB - yA)
        interArea = interW * interH
        areaInner = (inner[2] - inner[0]) * (inner[3] - inner[1])
        if areaInner == 0:
            return 0.0
        return interArea / areaInner

    def center_in_box(box, center):
        x, y = center
        return box[0] <= x <= box[2] and box[1] <= y <= box[3]

    merged = []
    by_class = {}
    for det in detections:
        by_class.setdefault(det[5], []).append(det)

    for cls, dets in by_class.items():
        n = len(dets)
        if n == 0:
            continue
        boxes = np.array([d[:4] for d in dets])
        dist = np.zeros((n, n), dtype=float)
        for i in range(n):
            for j in range(i + 1, n):
                val = iou(boxes[i], boxes[j])
                d = 1.0 - val
                dist[i, j] = dist[j, i] = d

        clustering = AgglomerativeClustering(
            n_clusters=None,
            affinity='precomputed',
            linkage=linkage,
            distance_threshold=1.0 - iou_threshold
        )
        labels = clustering.fit_predict(dist)

        for label in np.unique(labels):
            group = [dets[i] for i in range(n) if labels[i] == label]
            filtered = []
            for det in group:
                keep = True
                if containment_thresh:
                    for other in group:
                        if det is other:
                            continue
                        cont = containment_ratio(det[:4], other[:4])
                        if cont < containment_thresh and containment_ratio(other[:4], det[:4]) < containment_thresh:
                            keep = False
                            break
                if center_inclusion:
                    center = ((det[0] + det[2]) / 2, (det[1] + det[3]) / 2)
                    in_any = any(center_in_box(other[:4], center) for other in group if other is not det)
                    keep = keep and in_any
                if keep:
                    filtered.append(det)
            if not filtered:
                filtered = group

            xs = [d[0] for d in filtered] + [d[2] for d in filtered]
            ys = [d[1] for d in filtered] + [d[3] for d in filtered]
            max_conf = max(d[4] for d in filtered)
            merged.append((min(xs), min(ys), max(xs), max(ys), max_conf, cls))

    return merged


def draw_detections_varied_colors(image, detections, class_names, thickness=2, font_scale=0.5):
    annotated = image.copy()
    random.seed(42)

    for idx, det in enumerate(detections):
        x1, y1, x2, y2, conf, cls_id = det

        color = tuple(int(c) for c in np.random.randint(0, 256, size=3))

        label = f"{class_names[int(cls_id)]} {conf:.2f}"

        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        text_y = int(y1) - 5 if y1 > 10 else int(y1) + 15
        cv2.putText(
            annotated,
            label,
            (int(x1), text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            color,
            thickness=1,
            lineType=cv2.LINE_AA
        )

    return annotated

In [ ]:


image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048 
overlap = 1024
tiles, positions = tile_image(image, tile_size, overlap)

detections = detect_on_tiles(tiles, positions, tile_size , detectionModel)

class_names = detectionModel.names

annotated = draw_detections(image, detections, class_names)

cv2.imwrite("wynik_z_detections.png", annotated)
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(tiles[3], cv2.COLOR_BGR2RGB))
plt.title("Tile 0")
plt.show()

test_tile = tiles[3]
result = detectionModel(test_tile, conf=0.3)[0]
print("Wykryto bboxów:", len(result.boxes))

result.show()
import random

annotated = draw_detections_varied_colors(image, detections, class_names)
cv2.imwrite("wynik_rozrozne_kolory.png", annotated)